بسم الله

**Troisième extraction**

Troisième notebook. Le but est d'essayer d'extraire tous les livres du site, en une fois, sans passer par des sous catégories comme on l'a fait dans la première étape.

On utilisera cependant la même démarche pour récupérer les urls que dans fiqhqmaliki-extract-1/2.

## Etape 1 : Récupérer les urls

In [2]:
import requests
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup

In [36]:
##Aller chercher les urls des livres
urls = []
new_urls = []
to_supp =[]
NB_MAX_PAGES = 50
for i in range(2,NB_MAX_PAGES):
    if i%5 == 0:
        print('page ', i, '/', NB_MAX_PAGES, (i/NB_MAX_PAGES)*100, '%')
    i = str(i)
    r = requests.get('https://www.sifatusafwa.com/fr/accueil/?page=' + i)
    soup = BeautifulSoup(r.content, 'html.parser')
    s = soup.find('section', id = 'products')
    #s = s.find_all('div', class_ = 'products product_list row grid')
    #lines = s.find('h3', class_ = 's_title_block flex_child')
    #Affichage du contenu de la page
    #print(lines)

    #Extraire les liens
    
    for line in s.findAll('a'):
        urls.append(line.get('href'))#line.text.split(\'n') est une liste de chaine de caractères, donc [0] est 
        #seulement le str.
        #print(line.text.split('\n'))#line.text enlève les balises et les choses propres à html/css. Le .split split sur les \n
    #Tous les liens sont en double/ triple... on va les faire apparaitre qu'une seule fois.
    urls = list(np.unique(urls))
    #urls
    # On va supprimer les liens qui ne renvoient pas vers des livres : 
    #  - Ceux qui finissent pas '.asc' ou 'desc
    #  - Les liens de page ("?page=2", "?page=3"...) et le lien 'fiqh-maliki/'
    #  - les "javascript"
    
    #Suppression des liens ne renvoyant pas vers des livres.



page  5 / 50 10.0 %
page  10 / 50 20.0 %
page  15 / 50 30.0 %
page  20 / 50 40.0 %
page  25 / 50 50.0 %
page  30 / 50 60.0 %
page  35 / 50 70.0 %
page  40 / 50 80.0 %
page  45 / 50 90.0 %


In [37]:
len(urls)

635

In [38]:
for link in urls:
    if (link.endswith('asc'))==False and \
        (link.endswith('.desc'))==False and \
        len(link.split('/'))>5 and \
        (link.split('/')[5].startswith('?page=')) == False and \
        (link.startswith('https://www.sifatusafwa.com/fr/accueil/?rewrite'))== False:
        new_urls.append(link)            
    
for i in range(3):
    try:
        new_urls.remove('javascript:')
        new_urls.remove('javascript:;')
    except ValueError:
        continue
new_urls = pd.DataFrame(new_urls)
new_urls = new_urls.values.T.tolist()[0]
#on enlève tout ce qu'on a rajouté maintenant qu'on a pu faire la comparaison avec ce qu'il y avait
#a supprimer

In [39]:
len(new_urls)

575

## Etape 2 : Boucle 1 (par ouvrage)

In [54]:
urls = new_urls

In [40]:
url = urls[0]
r = requests.get(url)
soup = BeautifulSoup(r.content, 'html.parser')
s = soup.find('section', class_='product-features')
L_string = []
for text in s.findAll(class_ ='data-sheet flex_container'):
    L_string.append(text.text.split('\n'))
L_string = np.array(L_string)
df = pd.DataFrame((L_string))
df = df.drop_duplicates()#des fois les volumes apparaissent deux fois
df.columns = ['supp', 'categories', str(url), 'supp2']# a cette étape on remplace ce qu'il faut pour voir les sites en colonnes
df.drop(['supp', 'supp2'], axis = 1, inplace=True)
df.set_index('categories', inplace = True)
urls.remove(urls[0]) #On enlève le lien qu'on a déjà mis dans le dataframe de base


AttributeError: 'NoneType' object has no attribute 'findAll'

In [55]:
dfs = []
prices = []
auteurs = []
dates = []
domaines =[]
from tqdm import tqdm  
i = 0
for url in tqdm(urls):
    # Ajout des caractéristiques du livre (de fiqhmaliki-extract-1)
    i = i+1
    #print('url numero', i, '/', len(urls), (i/len(urls))*100, "%")
    if url.endswith('html'): #Pour pas que les javascript:: donnent une erreur
        r = requests.get(url)
    soup = BeautifulSoup(r.content, 'html.parser')
    s = soup.find('section', class_='product-features')
    L_string = []
    if s !=None:
        for text in s.findAll(class_ ='data-sheet flex_container'):
            L_string.append(text.text.split('\n'))
    else:
        print(url, "etape 1")
    L_string = np.array(L_string)
    df_other = pd.DataFrame((L_string))
    df_other = df_other.drop_duplicates()#des fois les volumes apparaissent deux fois
    if len(df_other.columns) == 4:
        df_other.columns = ['supp', 'categories',str(url), 'supp2']# a cette étape on remplace ce qu'il faut pour voir les sites en colonnes
        df_other.drop(['supp', 'supp2'], axis = 1, inplace=True)
        df_other.set_index('categories', inplace = True)
        dfs.append(df_other)
    else:
        print(url, 'etape 2')

  0%|          | 1/575 [00:04<39:50,  4.16s/it]

https://www.sifatusafwa.com/fr/accueil/ etape 1
https://www.sifatusafwa.com/fr/accueil/ etape 2


 24%|██▍       | 139/575 [04:32<13:55,  1.92s/it]

https://www.sifatusafwa.com/fr/déstockage-et-occasion/as-siyam-question-réponse-waqf-min-majmu-ah-zad.html etape 1
https://www.sifatusafwa.com/fr/déstockage-et-occasion/as-siyam-question-réponse-waqf-min-majmu-ah-zad.html etape 2


 49%|████▉     | 281/575 [09:10<07:51,  1.60s/it]

...

 79%|███████▉  | 453/575 [13:26<02:40,  1.31s/it]

https://www.sifatusafwa.com/fr/papeterie/stylo-bille-parker-gravé-dans-son-coffret-noirargent-calligraphie-sifatusafwa.html etape 1
https://www.sifatusafwa.com/fr/papeterie/stylo-bille-parker-gravé-dans-son-coffret-noirargent-calligraphie-sifatusafwa.html etape 2


 79%|███████▉  | 454/575 [13:27<02:59,  1.48s/it]

https://www.sifatusafwa.com/fr/papeterie/élégant-stylo-en-bois-avec-la-calligraphie-arabe-de-sifatusafwa.html etape 1
https://www.sifatusafwa.com/fr/papeterie/élégant-stylo-en-bois-avec-la-calligraphie-arabe-de-sifatusafwa.html etape 2


100%|██████████| 575/575 [16:34<00:00,  1.73s/it]


In [62]:
dfs_2 = []
from tqdm import tqdm  
prices = []
auteurs = []
dates = []
domaines = []
i = 0
for url in tqdm(urls):
    i = i+1
    #print('url numero', i, '/', len(urls), (i/len(urls))*100, "%")
    if url.endswith('html'): #Pour pas que les javascript:: donnent une erreur
        r = requests.get(url)
    soup = BeautifulSoup(r.content, 'html.parser')
    s = soup.find('section', id ='main')
## Ajout des auteurs/dates de décès/prix/domaines/descriptions (de fiqhmaliki-extract-2)
    #On vérifie si l'auteur du livre est disponible:
    s_auteur = s.find("a", class_= 'pro_extra_info_brand') #Si on divise pas en s_auteur et s_prix le find deviendra une liste car ce sera un find d'un find...
    
    #Soit pas d'auteur, soit auteur avec date, soit auteur sans date, soit auteur avec date mais deux
    #fois la parenthèse.

    if  s_auteur != None:#Auteur existe
        s_auteur = s_auteur.text         
        #Le try c'est juste pour couvrir tous les cas de date de décès possible
        try:#Cas 1 : La date existe : on l'ajoute
            s_dateauteur = s_auteur.split('(')[1].split('H)')[0]#s_auteur.split('(') = ['al qadhi iyyadh ', '544H)']
            dates.append(int(s_dateauteur))            

           #Ibn Rushd (Al-Jadd) (520H)	: split('\n') = ['', 'ibn rushd (Al hafid)', '']
        except IndexError:#Cas 2: Pas de date de décès disponible
            dates.append('') #On ne met pas de date pour pas qu'il y ait de décalage: un auteur avec la date de décès de l'auteur 
            #suivant
        except ValueError:#Cas 3: deux fois la parenthèse, il ne peut pas convertir en entier: c'est une 
            #chaine de caractères
            dates.append('') #Avantageux de remplir les auteurs après avoir vérifié la date car ibn rushd (al hafid) est coupé

        auteur = (s_auteur.split('\n')[1].split('(')[0])#s_auteur.split('\n) = ['', 'ibn rushd (Al hafid)', '']
         # ou ['', 'al qadi iyadd','(544)]
        auteurs.append(auteur) #Pour pas qu'on ait un espace à la fin.
            
    else: #Pas d'auteur
        auteurs.append('Auteur non disponible')
        dates.append('') #pour pas qu'il y ait de décalage: un auteur avec la date de décès de l'auteur suivant
    
    #Ajour du prix    
    s_prix = s.find("span", class_ = 'price')
    try:
        price = s_prix.text
        prices.append(price)

    except AttributeError: #s.find eset une liste et .text n'existe pas pour une liste: normalement plus d'erreur
        print(url, type(s))
    #Ajout du domaine:
    domaine = url.split('/')[4]
    domaines.append(domaine)

 54%|█████▎    | 309/575 [07:33<05:29,  1.24s/it]

https://www.sifatusafwa.com/fr/langue-arabe/al-moufrad-al-alam-ahmad-al-hachimi-imlae.html <class 'bs4.element.Tag'>


100%|██████████| 575/575 [13:35<00:00,  1.42s/it]


In [68]:
prices.append('')

On va faire un DataFrame avec les informations des livres (titre arabe, auteur, édition, volumes... de fiqh-extract1) et un autre avec les auteurs, les domaines.... (de fiqhmaliki-extract-2)

In [69]:
dfs_1 = pd.concat((i for i in dfs), axis = 1)
dfs_2 = pd.DataFrame({"lien": urls, 'auteurs': auteurs, 'date': dates, 'domaine':domaines, 'prix': prices})

In [70]:
dfs_1 = dfs_1.T


In [72]:
dfs_1['lien'] = dfs_1.index
dfs_1.set_index(np.arange(len(dfs_1)), inplace = True)

In [87]:
dfs_2['lien'] ==  dfs_1['lien']

ValueError: Can only compare identically-labeled Series objects

In [95]:
dfs_1.drop(['isbn', 'ean13'], axis = 1, inplace=True)#Données propres à l'édition d'un livre. Elles ont du
#etre présentes sur un des liens.

On pourra donc faire une jointure si besoin car les deux dataframes ont les mêmes sites aux mêmes index

On va faire un troisième DataFrame avec les descriptions des livres.

In [98]:
descriptions = []
descriptions_edition = []
i = 0
from tqdm import tqdm
for url in tqdm(urls):
    i = i+1
    #if i%10 == 0:
     #   print('url numero', i, '/', len(urls), (i/len(urls))*100, "%")
    if url.endswith('.html'):
        r = requests.get(url)
    soup = BeautifulSoup(r.content, 'html.parser')
    s = soup.find('section', id ='main')

    #On atteint le stade où les guillemets ne sont plus disponibles:
    # url = 'https://www.sifatusafwa.com/fr/fiqh-maliki/ikhtiyarat-al-hafidh-ibn-abdil-barr-al-fiqhiya.html'
    char = 1
    description = ""
    s_description = s.find(class_ = 'product-information')
    try:
        s_description = s_description.text.split('\n')
        while s_description[char] != "":
            description = description + s_description[char] + '\n' 
            char = char + 1
        descriptions.append(description)#On met entre crochets pour que plusieurs lignes = 1 élément
    except AttributeError as e:
        print('erreur url', url, e)#ces liens ne renvoient plus vers des livres, pas important

        
    #description d'édition: table avec juste ce qu'il faut car plusieurs editions pas toujours disponible, on fera une jointure sur le site si 
    # besoin. On met donc le site dans le tableau.
    
    s_editiondescription =  s.find(class_ = 'rte')
    if s_editiondescription!= None:
        descriptions_edition.append([s_editiondescription.text.split('\n')[0], url])

 54%|█████▎    | 309/575 [11:13<06:44,  1.52s/it]

erreur url https://www.sifatusafwa.com/fr/langue-arabe/al-moufrad-al-alam-ahmad-al-hachimi-imlae.html 'NoneType' object has no attribute 'text'


100%|██████████| 575/575 [18:33<00:00,  1.94s/it]


In [106]:
dfs_3 = pd.DataFrame({'description':descriptions, 'lien':urls})
dfs_3.head()

,description,lien
0,"Cet ouvrage, accessible et bénéfique à tout un...",https://www.sifatusafwa.com/fr/accueil/al-char...
1,Explication simplifiée du Matn Zad al-Moustaqn...,https://www.sifatusafwa.com/fr/accueil/al-iqna...
2,Le livre “Al-Iqna’a fi Masa’il al-Ijma’a” de l...,https://www.sifatusafwa.com/fr/accueil/al-matl...
3,Parmi les œuvres de Cheikh al-Islam Zakaria al...,https://www.sifatusafwa.com/fr/accueil/kitab-h...
4,"Cet ouvrage d'Ibn Abi Dounia, parle des djinns...",https://www.sifatusafwa.com/fr/actes-blamables...


In [108]:
dfs_2['lien'][1] == dfs_3['lien'][0]

True

On ajoute un dernier DataFrame avec les descriptions éditions (certains ouvrages ne possèdent qu'une seule édition donc beaucoup de valeurs nulles si on les met dans le même DataFrame)

In [109]:
dfs_editions = pd.DataFrame(descriptions_edition, columns=['description-edition', 'lien'])
dfs_editions.head()

,description-edition,lien
0,,https://www.sifatusafwa.com/fr/al-hakimiya-kou...
1,,https://www.sifatusafwa.com/fr/az-zuhd-wa-al-a...
2,,https://www.sifatusafwa.com/fr/biographies/mou...
3,,https://www.sifatusafwa.com/fr/déstockage-et-o...
4,,https://www.sifatusafwa.com/fr/déstockage-et-o...


In [110]:
dfs_1.to_csv('books_1.csv')
dfs_2.to_csv('books_2.csv')
dfs_3.to_csv('books_3.csv')

الحمد لله; la construction du dataset est terminée.